In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import random
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
trainData = '/kaggle/input/titanic/train.csv'
testData = '/kaggle/input/titanic/test.csv'
submissionExample = '/kaggle/input/titanic/gender_submission.csv'

data_train = pd.read_csv(trainData)

data_test = pd.read_csv(testData)

data_submissions = pd.read_csv(submissionExample)

data_test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [3]:
print(data_submissions)

     PassengerId  Survived
0            892         0
1            893         1
2            894         0
3            895         0
4            896         1
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         0

[418 rows x 2 columns]


In [4]:
nan_columns = data_test.isna().any()

print(nan_columns)

PassengerId    False
Pclass         False
Name           False
Sex            False
Age             True
SibSp          False
Parch          False
Ticket         False
Fare            True
Cabin           True
Embarked       False
dtype: bool


In [5]:
features = ['Pclass', 'Sex', 'Age', 'Fare', 'Cabin', 'Embarked']

x_train = data_train[features].copy()
x_test = data_test[features].copy()

y_train = data_train['Survived']

x_train['Sex'] = x_train['Sex'].replace({'male': 0, 'female': 1})
x_test['Sex'] = x_test['Sex'].replace({'male': 0, 'female': 1})

x_train['Embarked'] = x_train['Embarked'].replace({'S': 1, 'C': 2, 'Q': 3})
x_test['Embarked'] = x_test['Embarked'].replace({'S': 1, 'C': 2, 'Q': 3})

/tmp/ipykernel_18/3498157862.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  x_train['Sex'] = x_train['Sex'].replace({'male': 0, 'female': 1})
/tmp/ipykernel_18/3498157862.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  x_test['Sex'] = x_test['Sex'].replace({'male': 0, 'female': 1})
/tmp/ipykernel_18/3498157862.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future 

In [6]:
median_age = x_train['Age'].median()

x_train['Age'].fillna(median_age, inplace=True)
x_test['Age'].fillna(median_age, inplace=True)

FC_mean_fare = x_train.loc[x_train['Pclass'] == 1, 'Fare'].mean()
SC_mean_fare = x_train.loc[x_train['Pclass'] == 2, 'Fare'].mean()
TC_mean_fare = x_train.loc[x_train['Pclass'] == 3, 'Fare'].mean()

x_train.loc[(x_train['Pclass'] == 1) & (x_train['Fare'].isna()), 'Fare'] = FC_mean_fare
x_train.loc[(x_train['Pclass'] == 2) & (x_train['Fare'].isna()), 'Fare'] = SC_mean_fare
x_train.loc[(x_train['Pclass'] == 3) & (x_train['Fare'].isna()), 'Fare'] = TC_mean_fare

x_test.loc[(x_test['Pclass'] == 1) & (x_test['Fare'].isna()), 'Fare'] = FC_mean_fare
x_test.loc[(x_test['Pclass'] == 2) & (x_test['Fare'].isna()), 'Fare'] = SC_mean_fare
x_test.loc[(x_test['Pclass'] == 3) & (x_test['Fare'].isna()), 'Fare'] = TC_mean_fare

x_train['Cabin'].fillna(0, inplace=True)
x_test['Cabin'].fillna(0, inplace=True)

x_train.loc[x_train['Cabin'] != 0, 'Cabin'] = 1
x_test.loc[x_test['Cabin'] != 0, 'Cabin'] = 1

x_train['Embarked'].fillna(1, inplace=True)


/tmp/ipykernel_18/3771157943.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  x_train['Age'].fillna(median_age, inplace=True)
/tmp/ipykernel_18/3771157943.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using

In [7]:
nan_columns = x_train.isna().any()

print(nan_columns)

Pclass      False
Sex         False
Age         False
Fare        False
Cabin       False
Embarked    False
dtype: bool


In [8]:
model = RandomForestClassifier(random_state=1)

model.fit(x_train, y_train)

predictions = model.predict(x_test)

submission_df = pd.DataFrame({'PassengerId': data_test['PassengerId'], 'Survived': predictions})

submission_df.to_csv('submission.csv', index=False)

print(submission_df)

     PassengerId  Survived
0            892         0
1            893         0
2            894         0
3            895         1
4            896         0
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         0

[418 rows x 2 columns]
